# Кейс: Как выбрать помещение для октрытия кофейни?


**Области применения:**
- розничная торговля продуктами питания (FMCG)
- здравоохранение (аптеки, частные клиники)
- общепит (кафе, рестораны, бары)
- банковский сектор и страхование
- бьюти - индустрия (салоны красоты, магазины парфюмерии и косметики)
- DIY (строительные магазины)
- девелопмент торговых центров и др.

**Типовые задачи:**
- выбор оптимального размещения нового объекта
- оценка потенциального объема продаж, определение класса (масс-маркет, бизнес, люкс) и состава реализуемой продукции/услуг
- определение эффективного пути использования имеющегося объекта
- выбор оптимальной локации для размещения наружной рекламы и др. маркетинговых оффлайн активностей

Принципы масштабирования сети:
1. **Перехват трафика.** Этот принцип подразумевает открытие сетевых точек в зонах с высоким пешеходным трафиком (важно: высокий пешеходный трафик не всегда равен большому количеству целевой аудитории (ЦА) => анализируем состав ЦА), а также вблизи уже открытых точек конкурентов (важно: конкурентное преимущество);
2. **Синергия.** Эффект синергии достигается благодаря открытию смежных ниш бизнеса. Например: рядом с детскими товарами открывается магазин с товарами для дома/мам и пр.
3. **Доступность.** Принцип наименьших усилий (подробнее) - это   принцип, который основан на том, что человек по природе своей стремится приложить как можно меньше усилий для получения желаемого. При наличии различных возможностей  – клиент пойдёт туда, где ближе/привычнее/комфортнее. Критерии: радиус охвата, пешеходная доступность, транспортная доступность.
4. **Кластеризация.** Торговые точки должны быть кластеризованы (распределены на группы) как минимум по следующим категориям: бюджет района, тип населенного пункта (большой/малый, поселки и пр.), формат торговой точки. Это означает, что для каждой группы необходима индивидуальная стратегия масштабирования, ценообразования, ассортиментной политики и т.д.


In [5]:
import sys
print(sys.executable)

/Users/kolontay/opt/anaconda3/bin/python


In [13]:
# pip install h3
# pip install geopandas
# pip install osmnx

In [288]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
# Имопрт библиотек
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon

## Рандомный Hexagone в Краснодаре

In [243]:
def visualize_hexagons(hexagons, color="red", folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons: #индекс из списка индексов гексагонов
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False) #координаты гексагона для отрисовки
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline) # вообще добываем координаты гексагонов (список координатных линий для каждого гексагона)
    
    if folium_map is None:
        # создается карта (m) на основе вершин гексагонов
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')
    else:
        m = folium_map
     # добавляются гексагоны на карту   
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
  
h3_address = h3.geo_to_h3(45.035470, 38.975313,  9) # 9 - индекс, определяющий размер гексагона                                                                                                     
visualize_hexagons([h3_address])

## Возьмем полигон в Москве

In [559]:
def visualize_polygons(geometry):
    # получаем координаты центроидов
    lats, lons = get_lat_lon(geometry) 
    #создаем карту
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')

    # преобразование в GeoJSON
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    
    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon
  
# выгрузим границы Краснодара из OSM
cities = ['Москва, Россия']
#cities = ['Москва, Россия']
polygon_krd = ox.features_from_place(cities, {'boundary':'administrative'}).reset_index()
polygon_krd = polygon_krd[(polygon_krd['name'] == 'Юго-Восточный административный округ')]
# посмотрим что получилось
visualize_polygons(polygon_krd['geometry'])

/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [560]:
# Для корректной обработки возьмем один полигон из MultiPolygon

In [562]:
from shapely.geometry import Polygon, MultiPolygon

def extract_largest_polygon(geometry):
    # Проверяем тип геометрии
    if isinstance(geometry, MultiPolygon):
        # Извлекаем полигон с наибольшей площадью из MultiPolygon
        largest_polygon = max(geometry.geoms, key=lambda a: a.area)
    elif isinstance(geometry, Polygon):
        # Если это уже полигон, просто возвращаем его
        largest_polygon = geometry
    else:
        # Если тип не поддерживается
        raise ValueError("Unsupported geometry type")
    return largest_polygon

# Применение функции к каждой геометрии в GeoDataFrame
polygon_krd['geometry'] = polygon_krd['geometry'].apply(extract_largest_polygon)

## Заполним полином гексагонами 

In [573]:
def create_hexagons(geoJson):
    
    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 8))
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')
        m.add_child(my_PolyLine)
        
    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])
        
    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))
        
    return m, polygons_hex, polylines
# polygon_hex , polylines - геометрии гексагонов в разных форматах

# сгенерим гексагоны внутри полигона 
geoJson = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
geoJson = geoJson['features'][0]['geometry']
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1],
                                                      np.array(geoJson['coordinates'][0])[:, 0])).tolist()]}

m, polygons, polylines = create_hexagons(geoJson)
m

## Выгружаем данные OpenStreetMap (OSM)  по тегам из заданного города

In [577]:
def osm_query(tag, city):
    gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    print(gdf.shape)
    return gdf
  
 # Выгрузим интересующие нас категории объектов 
tags = [
        {'building' : 'apartments'}, {'building' : 'detached'}, 
        {'building' : 'dormitory'}, {'building' : 'hotel'}, 
        {'building' : 'house'}, {'building' : 'semidetached_house'}, 
        {'building' : 'terrace'},  {'building' : 'commercial'},
        {'building' : 'office'},  {'building' : 'terrace'},  
        {'building' : 'terrace'}, {'building':'retail'}, 
        {'building':'train_station'},
        {'highway' : 'bus_stop'}, {'footway':'crossing'},
        {'amenity':'cafe'}, {'amenity':'fast_food'}, 
        {'amenity':'restaurant'}, {'amenity':'college'}, 
        {'amenity':'language_school'},  {'amenity':'school'},  
        {'amenity':'university'},  {'amenity':'atm'},  
        {'amenity':'bank'},  {'amenity':'clinic'},  
        {'amenity':'hospital'},  {'amenity':'pharmacy'},  
        {'amenity':'theatre'},  {'amenity':'townhall'},  
        {'amenity':'bench'}, 
       ]
cities = ['Москва, Россия']

gdfs = []
for city in cities:
    for tag in tags:
        gdfs.append(osm_query(tag, city))
        
# посмотрим что получилось
data_poi = pd.concat(gdfs)
data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

# добавим координаты/центроиды
lat, lon = get_lat_lon(data_poi['geometry'])
data_poi['lat'] = lat
data_poi['lon'] = lon

/tmp/ipykernel_11017/2252971327.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии


(35769, 4)


/tmp/ipykernel_11017/2252971327.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии


(1558, 4)


/tmp/ipykernel_11017/2252971327.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии


...

/tmp/ipykernel_11017/2252971327.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии


(47, 4)


/tmp/ipykernel_11017/2252971327.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index() # делает запрос к OSM, извлекает геометрии


(22791, 4)


/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [576]:
data_poi

,city,object,type,geometry,lat,lon
0,Москва,building,apartments,"POLYGON ((37.22605 55.51590, 37.22610 55.51601...",55.515882,37.226577
1,Москва,building,apartments,"POLYGON ((37.22435 55.51618, 37.22440 55.51629...",55.516155,37.224920
2,Москва,building,apartments,"POLYGON ((37.22453 55.51664, 37.22459 55.51675...",55.516603,37.225122
3,Москва,building,apartments,"POLYGON ((37.22477 55.51708, 37.22482 55.51719...",55.517047,37.225350
4,Москва,building,apartments,"POLYGON ((37.29724 55.48611, 37.29828 55.48612...",55.486054,37.297759
...,...,...,...,...,...,...
22786,Москва,amenity,bench,"LINESTRING (37.45122 55.52537, 37.45122 55.52527)",55.525321,37.451219
22787,Москва,amenity,bench,"POLYGON ((37.69821 55.58234, 37.69818 55.58233...",55.582308,37.698235
22788,Москва,amenity,bench,"POLYGON ((37.69798 55.58223, 37.69795 55.58221...",55.582192,37.698005
22789,Москва,amenity,bench,"LINESTRING (37.69799 55.58205, 37.69797 55.582...",55.582108,37.697925


## Джоин данных с гексагонами 

In [581]:
# sjoin - spatial join - пересекаем гексагоны с объектами (определяем какие объекты находятся в разрезе каждого гексагона)

gdf_1 = gpd.GeoDataFrame(data_poi, geometry=gpd.points_from_xy(data_poi.lon, data_poi.lat))

gdf_2 = pd.DataFrame(polygons, columns = ['geometry'])
gdf_2['polylines'] = polylines
gdf_2['geometry'] = gdf_2['geometry'].astype(str)
geometry_uniq = pd.DataFrame(gdf_2['geometry'].drop_duplicates())
geometry_uniq['id'] = np.arange(len(geometry_uniq)).astype(str)
gdf_2 = gdf_2.merge(geometry_uniq, on = 'geometry')
gdf_2['geometry'] = gdf_2['geometry'].apply(wkt.loads)
gdf_2 = gpd.GeoDataFrame(gdf_2, geometry='geometry')

itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')
itog_table = itog_table.dropna()
itog_table.rename(columns={'index_right': 'object_id'}, inplace=True)
print (itog_table.shape[0])
itog_table.head(3)

10308


/Users/kolontay/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_11017/2604135154.py:14: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')


,geometry,polylines,id,object_id,city,object,type,lat,lon
0,"POLYGON ((37.72827 55.72097, 37.73618 55.72143...","[(55.72096888443625, 37.728272933100605), (55....",0,272.0,Москва,amenity,cafe,55.721351,37.733797
0,"POLYGON ((37.72827 55.72097, 37.73618 55.72143...","[(55.72096888443625, 37.728272933100605), (55....",0,586.0,Москва,building,commercial,55.721576,37.732252
0,"POLYGON ((37.72827 55.72097, 37.73618 55.72143...","[(55.72096888443625, 37.728272933100605), (55....",0,11161.0,Москва,building,apartments,55.721683,37.736027


## Количество кафе

In [582]:
def create_choropleth(data, json, columns, legend_name, feature, bins):
    
    lat, lon = get_lat_lon(data['geometry'])

    m = folium.Map(location=[sum(lat)/len(lat), sum(lon)/len(lon)], zoom_start=13, tiles='cartodbpositron')
    
    folium.Choropleth(
        geo_data=json,
        name="choropleth",
        data=data,
        columns=columns,
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_name,
        nan_fill_color = 'black',
        bins = bins

    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m
  
# подготовим данные 
itog_table['geometry'] = itog_table['geometry'].astype(str) #для groupby
itog_table['id'] = itog_table['id'].astype(str) #для Choropleth
agg_all = itog_table.groupby(['geometry','type','id'], as_index = False).agg({'lat':'count'}).rename(columns = {'lat':'counts'})
agg_all['geometry'] = agg_all['geometry'].apply(wkt.loads) #возвращаем формат геометрий

agg_all_cafe = agg_all.query("type == 'cafe'")[["geometry","counts",'id']]
agg_all_cafe['id'] = agg_all_cafe['id'].astype(str)
data_geo_1 = gpd.GeoSeries(agg_all_cafe.set_index('id')["geometry"]).to_json()

create_choropleth(agg_all_cafe, data_geo_1, ["id","counts"], 'Cafe counts', 'counts', 5)
    

/tmp/ipykernel_11017/1016455783.py:27: UserWarning: Geometry column does not contain geometry.
  itog_table['geometry'] = itog_table['geometry'].astype(str) #для groupby
/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


## Добавим данные по этажам

In [583]:
gdf_aparts = ox.geometries_from_place(city, {'building' : 'apartments'}).reset_index()

/tmp/ipykernel_11017/3271568645.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_aparts = ox.geometries_from_place(city, {'building' : 'apartments'}).reset_index()


In [584]:
# достанем координаты из геометрии
lat_g, lon_g = get_lat_lon(gdf_aparts['geometry'])
gdf_aparts['lat'] = lat_g
gdf_aparts['lon'] = lon_g

/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [589]:
gdf_aparts = gpd.GeoDataFrame(gdf_aparts, geometry=gpd.points_from_xy(gdf_aparts.lon, gdf_aparts.lat))

In [591]:
# Преобразование строки WKT в объекты Shapely
itog_table['geometry'] = itog_table['geometry'].apply(wkt.loads)

my_table = gpd.sjoin(itog_table, gdf_aparts[['geometry','building:levels']], how='left', predicate='intersects')
my_table = my_table.drop_duplicates(subset=['object_id', 'object', 'type', 'lat', 'lon'])

/tmp/ipykernel_11017/2141873479.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  my_table = gpd.sjoin(itog_table, gdf_aparts[['geometry','building:levels']], how='left', predicate='intersects')


In [595]:
# Расчитаем количество проживающих людей
apartments = ['apartments' , 'dormitory']
houses = ['house', 'semidetached_house', 'detached', 'terrace']
aparts = my_table[(my_table['type'].isin(apartments))|(my_table['type'].isin(houses))]
aparts['people_counts'] = 'not_living_area'
aparts.loc[aparts['type'].isin(apartments),'people_counts'] = aparts.loc[aparts['type'].isin(apartments),'building:levels'].astype(float)*10*3
aparts.loc[aparts['type'].isin(houses),'people_counts'] = aparts.loc[aparts['type'].isin(houses),'building:levels'].astype(float)*3

/Users/kolontay/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [598]:
def create_heatmap(data, lat_lon_feature):
    
    m = folium.Map(location=[sum(data['lat'])/len(data['lat']), sum(data['lon'])/len(data['lon'])], zoom_start=13, tiles='cartodbpositron')
    
    HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(), 
                    radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
    return m
  
# карта плотности населения
create_heatmap(aparts, ['lat', 'lon', 'people_counts'])

/tmp/ipykernel_11017/634780811.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
/tmp/ipykernel_11017/634780811.py:5: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(),


In [602]:
group_people = aparts.groupby(['geometry','id'], as_index = False).agg(peopls = ('people_counts','sum'))
data_geo_2 = gpd.GeoSeries(group_people.set_index('id')["geometry"]).to_json()
create_choropleth(group_people, data_geo_2, ["id","peopls"], 'people counts', 'counts', 10)

/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [603]:
group_people

,geometry,id,peopls
0,"POLYGON ((37.68360 55.65560, 37.68800 55.65160...",69,0
1,"POLYGON ((37.70030 55.64820, 37.70820 55.64860...",161,10215.0
2,"POLYGON ((37.69940 55.65650, 37.70730 55.65690...",56,1200.0
3,"POLYGON ((37.70730 55.65690, 37.71170 55.65300...",120,2970.0
4,"POLYGON ((37.72050 55.64510, 37.72400 55.64950...",17,10800.0
...,...,...,...
120,"POLYGON ((37.79330 55.63260, 37.78540 55.63210...",48,4800.0
121,"POLYGON ((37.79690 55.63690, 37.79250 55.64090...",27,4680.0
122,"POLYGON ((37.78460 55.64040, 37.79250 55.64090...",53,1050.0
123,"POLYGON ((37.79250 55.64090, 37.79690 55.63690...",71,2970.0


In [609]:
check = agg_all_cafe.merge(group_people, on = 'geometry')

In [611]:
check['val'] = check['peopls']/check['counts']

In [613]:
data_geo_3 = gpd.GeoSeries(check.set_index('id_x')["geometry"]).to_json()
create_choropleth(check, data_geo_3, ["id_x","val"], 'people/cafe', 'val', 10)

/tmp/ipykernel_11017/931173506.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/tmp/ipykernel_11017/931173506.py:17: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
